<a href="https://colab.research.google.com/github/aishstronomer/flare-finder/blob/main/solar_image_flare_labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

to-do:
 - store solar_images big-flare labels in a file: big_flare_labels.csv

In [1]:
# set globals

# Do Google Colab things
try:
    from google.colab import drive

    drive.mount("/content/drive", force_remount=True)
    IN_COLAB = True
except:
    IN_COLAB = False

# standard imports
import glob
import os
import pandas as pd
import sys

In [2]:
# we want a function get_is_big_flare() that has the following schema:
# - input: solar_image_path
# - output: is_big_flare (bool)


def get_is_big_flare(solar_image_path, big_flare_max_dttimes, tolerance):

    is_big_flare = False

    # get datetime of solar_image_path: solar_image_datetime
    solar_image_fname = os.path.basename(solar_image_path)
    solar_image_datetime = pd.to_datetime(
        solar_image_fname[0 : solar_image_fname.rfind(".png")]
    ).replace(tzinfo=None)

    # compare solar_image_datetime with max_datetimes and set is_big_flare to True
    # if solar_image_datetime within some tolerance of max_datetimes
    min_time_delta = min(abs(solar_image_datetime - big_flare_max_dttimes))
    if min_time_delta < tolerance:
        is_big_flare = True

    return is_big_flare


# get big_flare_max_dttimes
path_to_coderepo = (
    "/content/drive/MyDrive/ML_project/code_repo/flare-finder" if IN_COLAB else "."
)
goes_file_path = f"{path_to_coderepo}/goes_events_clean_2015.csv"
goes_events_df = pd.read_csv(goes_file_path)
goes_mx_events = goes_events_df[
    goes_events_df["Particulars_a"].str.lower().str.startswith(("m", "x"))
]
big_flare_max_dttimes = pd.to_datetime(goes_mx_events["max_datetime"]).dropna()

get_is_big_flare(
    f"{path_to_coderepo}/../../data/sdo_images/2015-01-01 00:00:12.340000+00:00.png",
    big_flare_max_dttimes,
    pd.to_timedelta(10, "min"),
)

False

In [3]:
# get labels for solar images and store in big_flare_labels.csv


def get_big_flare_labels(solar_images_folder, goes_file_path, big_flare_time_tolerance):
    big_flare_labels_df = pd.DataFrame(
        columns=["solar_image_filename", "is_big_flare", "time_tolerance_minutes"]
    )

    # get big_flare_max_dttimes
    goes_events_df = pd.read_csv(goes_file_path)
    goes_mx_events = goes_events_df[
        goes_events_df["Particulars_a"].str.lower().str.startswith(("m", "x"))
    ]
    big_flare_max_dttimes = pd.to_datetime(goes_mx_events["max_datetime"]).dropna()

    # get solar_image_paths
    solar_image_paths = glob.glob(f"{solar_images_folder}/*png")

    # for each solar image path, get is_big_flare and populate df
    time_tolerance_minutes = big_flare_time_tolerance.total_seconds() / 60
    for i, solar_image_path in enumerate(solar_image_paths[0:2]):
        is_big_flare = get_is_big_flare(
            solar_image_path, big_flare_max_dttimes, big_flare_time_tolerance
        )
        big_flare_labels_df.loc[len(big_flare_labels_df)] = (
            os.path.basename(solar_image_path),
            is_big_flare,
            time_tolerance_minutes,
        )

    return big_flare_labels_df


goes_file_path = f"{path_to_coderepo}/goes_events_clean_2015.csv"
solar_images_folder = f"{path_to_coderepo}/../../data/sdo_images"

big_flare_labels_df = get_big_flare_labels(
    solar_images_folder, goes_file_path, pd.to_timedelta(10, "min")
)
big_flare_labels_df.to_csv(f"{solar_images_folder}/big_flare_labels.csv", index=False)
pd.read_csv(f"{solar_images_folder}/big_flare_labels.csv")

,solar_image_filename,is_big_flare,time_tolerance_minutes
0,2015-01-08 07:00:12.340000+00:00.png,False,10.0
1,2015-04-03 23:00:12.340000+00:00.png,False,10.0
